# MLP classifier for mutliclass classifications

In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

import json
import numpy as np
import re

## Formatting datas



**Labeled datas :** The labels of the datas are the following from the json file is :
[left_hand_class, right_hand_class] or [both_hands_class ]

with the following labels : 
```py
LABELS_MAPPING = {
    "heart_1": 0,
    "heart_2": 1,
    "heart_3": 2,
    "index_lifted": 3,
    "f*ck_sign": 4,
    "victory_sign": 5,
    "ok": 6,
    "fist": 7,
    "middle_finger_touching_thumb": 8,
    "little_finger_up": 9,
    "ring_finger_touching_thumb": 10,
    "triangle": 11,
    "thumbs_up": 12,
}
```

**Format for MLP prediction:** The MLP model does handle multi-class prediction. The output format will be as followed : `[label_0_bool, label_2:bool, label_3:bool,...]`


In [2]:
label_file = '../data/labels.json'

with open(label_file, 'r') as f:
    label_data = json.load(f)['labels']
label_data[1:3]

[{'path': './data/images_test/label_9_image_10.jpg', 'label': [4]},
 {'path': './data/images_test/label_8_image_1.jpg', 'label': None}]

In [3]:
len(label_data)

240

In [4]:
LABELS_MAPPING = {
    "heart_1": 0,
    "heart_2": 1,
    "heart_3": 2,
    "index_lifted": 3,
    "f*ck_sign": 4,
    "victory_sign": 5,
    "ok": 6,
    "fist": 7,
    "middle_finger_touching_thumb": 8,
    "little_finger_up": 9,
    "ring_finger_touching_thumb": 10,
    "triangle": 11,
    "thmubs_up":12,
    'None':None
}


In [5]:
# getting X and y
X = []
y = []
labels = {str(i): 0 for i in range(len(LABELS_MAPPING))}
labels['None'] = 0

for img in label_data:
    nb = re.findall("\d+", img['path'])
    landmarks = np.load(f'../data/landmarks/label_{nb[0]}_image_{nb[1]}.npy')
    
    img['label'] = [None] if img['label'] is None else img['label'] # check
    arity = True if len(img['label']) == 1 else False # arity

    if np.isnan(landmarks[0][0][0]) == False:
        X.append(landmarks[0])
        y.append([img['label'][0] == j for j in range(len(LABELS_MAPPING))]) # left_hand_label
        if img['label'][0] is None :
            labels['None'] += 1
        else :
            labels[str(img['label'][0])] += 1
    if np.isnan(landmarks[1][0][0]) == False:
        X.append(landmarks[1])
        y.append([img['label'][0] == j for j in range(len(LABELS_MAPPING))] if arity else [img['label'][1] == j for j in range(len(LABELS_MAPPING))]) # right_hand_label
        lab = img['label'][0] if arity else img['label'][1]
        if lab is None :
            labels['None'] += 1
        else :
            labels[str(lab)] += 1



X = np.array(X)                 # (N, 2, 21, 3)
X = X.reshape(len(X), -1)       # flatten the array => (N, 2, 63)
len(X)

432

## Preprocessing
A preprocessing on the X data could improve the model accuracy very efficiently. The idea would be to apply some transformations on the hand's coordinates.

## Training the model

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                    random_state=1, train_size=0.8)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPClassifier(random_state=1, max_iter=300)
mlp.fit(X_train, y_train)

/Users/louise_monciero/Bureau-no-icloud/dev/HandTracker/.venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=300, random_state=1)

## Evaluate the model

In [7]:
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 82.76%


In [8]:
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.83      0.77         6
           1       1.00      0.75      0.86         8
           2       1.00      0.75      0.86         8
           3       1.00      0.71      0.83         7
           4       0.86      1.00      0.92         6
           5       1.00      0.60      0.75         5
           6       1.00      0.86      0.92         7
           7       0.00      0.00      0.00         0
           8       0.80      1.00      0.89         8
           9       1.00      0.86      0.92         7
          10       1.00      0.88      0.93         8
          11       0.86      1.00      0.92         6
          12       1.00      1.00      1.00         7
          13       0.00      0.00      0.00         0

   micro avg       0.92      0.86      0.89        83
   macro avg       0.80      0.73      0.76        83
weighted avg       0.94      0.86      0.89        83
 s

/Users/louise_monciero/Bureau-no-icloud/dev/HandTracker/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/louise_monciero/Bureau-no-icloud/dev/HandTracker/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/louise_monciero/Bureau-no-icloud/dev/HandTracker/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parame

In [9]:
import pandas as pd
import plotly.express as px

# Data ----
steps = [1, 2, 3, 4, 5]
accuracy = [16.67, 45.83, 50.00, 63.16, 82.76] # (%)
descriptions = [
    "189 labeled hands | train_size = 0.25 ",
    "189 labeled hands | train_size = 0.25 | StandardScaler()",
    "189 labeled hands | train_size = 0.80 | StandardScaler()",
    "189 labeled hands | train_size = 0.80 | StandardScaler() | stratify=y",
    "432 labeled hands | train_size = 0.80 | StandardScaler() | stratify=y"
]

df = pd.DataFrame({
    "step": steps,
    "accuracy": accuracy,
    "description": descriptions
})

# Plot --------
fig = px.line(
    df,
    x="step",
    y="accuracy",
    markers=True,
    title="Accuracy improvement over training steps",
    hover_data={
        "step": True,
        "accuracy": ":.2f",
        "description": True
    }
)

fig.update_layout(
    xaxis_title="Step",
    yaxis_title="Accuracy (%)",
    xaxis=dict(dtick=1),
    hoverlabel=dict(bgcolor="white")
)

# Saving ---
fig.write_html(
    "results/accuracy_steps.html",
    include_plotlyjs="cdn",
    full_html=True
)

ModuleNotFoundError: No module named 'pandas'

## Saving

In [10]:
import pickle

with open('mlp_classifier.pkl', 'wb') as f:
    pickle.dump(mlp, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)